<h2>Data Collection</h2>

Data was downloaded from the US Department of Agriculture

https://www.fs.usda.gov/rds/archive/catalog/RDS-2013-0009.6

The data was provided as a SQLite database file. This was opened in DB Viewer to examine its structure. The data used in this work is in a single table so uneeded columns were removed. The table was then exported as a .csv file.

None of the data contains any personal identification information. The data was made available by its authors for use without additonal permission or fees.

Short, Karen C. 2022. Spatial wildfire occurrence data for the United States, 1992-2020 [FPA_FOD_20221014]. 6th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.6




In [1]:
# Import dependencies

import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input
from base64 import b64encode
import io

In [2]:
## Read in csv file
df = pd.read_csv('Fires.csv')

<h2>Data Cleaning</h2>

In [3]:
## Some of the Fire Causes are too long & are causing problems with bar graph formating 


## Replace longer names with shorter ones
df['NWCG_GENERAL_CAUSE'].replace(
        {'Debris and open burning': 'Debris Burning', 
         'Missing data/not specified/undetermined': 'Undetermined', 
         'Recreation and ceremony': 'Recreation',
         'Equipment and vehicle use': 'Equipment use',
         'Arson/incendiarism': 'Arson',
         'Railroad operations and maintenance': 'Railroad',
         'Misuse of fire by a minor': 'Children',
         'Firearms and explosives use': 'Firearms',
         'Power generation/transmission/distribution': 'Electricity supply'
         }, inplace=True)



<h2>Graphing Data using Plotly</h2>

In [10]:
# Graph of number of fires per year for the ten states with the largest numbers of fires

# Selecting only columns needed from imported data
fire_years_state_df = df[['OBJECTID','FIRE_YEAR','STATE']]

# Setting up variables for each state, with fire data grouped by year
national_graph_data = fire_years_state_df.groupby('FIRE_YEAR').count()
ca_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'CA')].groupby('FIRE_YEAR').count()
ga_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'GA')].groupby('FIRE_YEAR').count()
tx_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'TX')].groupby('FIRE_YEAR').count()
nc_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'NC')].groupby('FIRE_YEAR').count()
fl_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'FL')].groupby('FIRE_YEAR').count()
sc_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'SC')].groupby('FIRE_YEAR').count()
ny_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'NY')].groupby('FIRE_YEAR').count()
ms_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'MS')].groupby('FIRE_YEAR').count()
az_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'AZ')].groupby('FIRE_YEAR').count()
al_graph_data = fire_years_state_df[(fire_years_state_df['STATE']== 'AL')].groupby('FIRE_YEAR').count()

# Set up initial graph showing the number of fires per year nationally
fig_line = px.line(national_graph_data, labels={
                     "FIRE_YEAR": "Year",
                     "value": "Number of Fires"},
                title='Fires by Year: United States'
            )
# Remove the default legend
fig_line.update_layout(showlegend = False)

# Changing graph layout to include buttons which cause different data to be displayed in the graph
fig_line.update_layout(
    updatemenus=[
        
        # displays buttons on graph
        dict(type="dropdown",
            direction="down",
            x=0.2,
            y=1.15,
            showactive=True,
            # List of buttons to be added. One dictionary for each button
            buttons=list(
                [
                dict(
                    # Text on button
                    label= 'United States',
                    method="update",
                    # Change to make when button is selected
                    args=[{"y" : [national_graph_data.OBJECTID]},
                          {"title": "Fires by Year: United States"}
                          ]),
                
                dict(
                    label= 'CA',
                    method="update",
                    args=[{"y" : [ca_graph_data.OBJECTID]},
                          {"title": "Fires by Year: California"}
                          ]),
                
                dict(
                    label= 'GA',
                    method="update",
                    args=[{"y" : [ga_graph_data.OBJECTID]},
                          {"title": "Fires by Year: Georgia"}
                          ]),
                
                dict(
                    label= 'TX',
                    method="update",
                    args=[{"y" : [tx_graph_data.OBJECTID]},
                         {"title": "Fires Causes by Year: Texas"}
                         ]),
                
                dict(
                    label= 'NC',
                    method="update",
                    args=[{"y" : [nc_graph_data.OBJECTID]},
                          {"title": "Fires by Year: North Carolina"}
                          ]),
                
                dict(
                    label= 'FL',
                    method="update",
                    args=[{"y" : [fl_graph_data.OBJECTID]},
                         {"title": "Fires by Year: Florida"}
                         ]),
                
                dict(
                    label= 'SC',
                    method="update",
                    args=[{"y" : [sc_graph_data.OBJECTID]},
                         {"title": "Fires by Year: South Carolina"}
                         ]),
                
                dict(
                    label='NY',
                    method="update",
                    args=[{"y" : [ny_graph_data.OBJECTID]},
                         {"title": "Fires by Year: New York"}
                         ]),
                
                dict(
                    label= 'MS',
                    method="update",
                    args=[{"y" : [ms_graph_data.OBJECTID]},
                         {"title": "Fires by Year: Mississippi"}
                         ]),
                
                dict(
                    label='AZ',
                    method="update",
                    args=[{"y" : [az_graph_data.OBJECTID]},
                         {"title": "Fires by Year: Arizona"}
                         ]),
                
                dict(
                    label= 'AL',
                    method="update",
                    args=[{"y" : [al_graph_data.OBJECTID]},
                         {"title": "Fires by Year: Alabama"}
                         ]),
                
                
                
                ]
            ))                    
        ]
    )



In [11]:
# Bar Graph of fire causes per state and nationally for the ten states with the largest numbers of fires

# Selecting only columns needed from imported data
fire_cause_df = df[['OBJECTID','NWCG_GENERAL_CAUSE','STATE']]

# Setting up variables for each state, with imported data filtered by state, group by fire cause and counted 
national_cause = fire_cause_df.groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
ca_cause = fire_cause_df[(fire_cause_df['STATE']== 'CA')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
ga_cause = fire_cause_df[(fire_cause_df['STATE']== 'GA')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
tx_cause = fire_cause_df[(fire_cause_df['STATE']== 'TX')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
nc_cause = fire_cause_df[(fire_cause_df['STATE']== 'NC')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
fl_cause = fire_cause_df[(fire_cause_df['STATE']== 'FL')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
sc_cause = fire_cause_df[(fire_cause_df['STATE']== 'SC')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
ny_cause = fire_cause_df[(fire_cause_df['STATE']== 'NY')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
ms_cause = fire_cause_df[(fire_cause_df['STATE']== 'MS')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
az_cause = fire_cause_df[(fire_cause_df['STATE']== 'AZ')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)
al_cause = fire_cause_df[(fire_cause_df['STATE']== 'AL')].groupby('NWCG_GENERAL_CAUSE').count().drop('STATE', axis=1)

# Set up initial bar graph showing the fire causes per state or nationally
fig_bar = px.bar(national_cause, labels={
                     "NWCG_GENERAL_CAUSE": "Cause of Fire",
                     "value": "Number of Fires"},
                title='Fires Causes by Year: United States'
                )

# Changing graph layout to include buttons which cause different data to be displayed in the graph
fig_bar.update_layout(
    updatemenus=[
        # displays buttons on graph
        dict(type="dropdown",
            direction="down",
            x=0.2,
            y=1.15,
            showactive=True,
            # List of buttons to be added. One dictionary for each button
            buttons=list(
                [
                dict(
                    # Text on button
                    label= 'United States',
                    method="update",
                    # Change to make when button is selected
                    args=[{"y" : [national_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: United States"}
                         ]),
                
                dict(
                    label= 'CA',
                    method="update",
                    args=[{"y" : [ca_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: California"}
                         ]),
                
                dict(
                    label= 'GA',
                    method="update",
                    args=[{"y" : [ga_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Georgia"}
                         ]),
                
                dict(
                    label= 'TX',
                    method="update",
                    args=[{"y" : [tx_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Texas"}
                         ]),
                
                dict(
                    label= 'NC',
                    method="update",
                    args=[{"y" : [nc_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: North Carolina"}
                         ]),
                
                dict(
                    label= 'FL',
                    method="update",
                    args=[{"y" : [fl_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Florida"}
                         ]),
                
                dict(
                    label= 'SC',
                    method="update",
                    args=[{"y" : [sc_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: South Carolina"}
                         ]),
                dict(
                    label= 'NY',
                    method="update",
                    args=[{"y" : [ny_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: New York"}
                         ]),
                
                dict(
                    label= 'MS',
                    method="update",
                    args=[{"y" : [ms_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Mississippi"}
                         ]),
                
                dict(
                    label= 'AZ',
                    method="update",
                    args=[{"y" : [az_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Arizona"}
                         ]),
                
                dict(
                    label= 'AL',
                    method="update",
                    args=[{"y" : [al_cause.OBJECTID]},
                         {"title": "Fires Causes by Year: Alabama"}
                         ]),
                                ]
            ))                    
        ]
    )                     

# Remove default legend, sort bars in descending order
fig_bar.update_layout(showlegend=False, xaxis={'categoryorder':'total descending'})

<h2>Exporting Graphs using Dash</h2>

In [12]:
## Uses dash to export both graphs as a single html file

app = Dash(__name__)

buffer = io.StringIO()

fig_line.write_html(buffer)
fig_bar.write_html(buffer)

html_bytes = buffer.getvalue().encode()
encoded = b64encode(html_bytes).decode()

app.layout = html.Div([
    html.H2('Wildfire Dashboard'),
    dcc.Graph(id="graph1", figure=fig_line),
    dcc.Graph(id="graph2", figure=fig_bar),
    html.A(
        html.Button("Download as HTML"), 
        id="download",
        href="data:text/html;base64," + encoded,
        download="index.html"
    )
])

app.run_server(debug=True)

In [7]:
## DON"T FORGET TO ADD THIS TO INDEX>HTML BEFORE COMMITING TO GITHUB
## goes between <body> and <div>
## <h1><center>Wildfire Dashboard</center></h1>

<h2>Final dashboard is deployed to:</h2>

<h3>https://jlawrence16.github.io/wildfire_project/</h3>